In [ ]:
import numpy as np
import pandas as pd
import os
import geopandas as gpd
from shapely.wkt import loads
from city_road_network.utils.utils import get_data_subdir, get_html_subdir
import time
import pickle
from city_road_network.algo.naive_sim import run_naive_simulation
from city_road_network.utils.map import draw_trips_map
from city_road_network.utils.io import read_graph
from city_road_network.config import default_crs

In [ ]:
city_name = "saint_petersburg"
data_dir = get_data_subdir(city_name)
html_dir = get_html_subdir(city_name)

In [ ]:
G = read_graph(os.path.join(data_dir, "nodelist_upd.csv"), os.path.join(data_dir, "edgelist_upd.csv"))

In [ ]:
trip_mat = np.load(os.path.join(data_dir, "trip_mat.npy"))
n = trip_mat.shape[0]

In [ ]:
zones_df = pd.read_csv(os.path.join(data_dir, "zones_upd.csv"), index_col=0)
zones_df["geometry"] = zones_df["geometry"].apply(loads)
zones_gdf = gpd.GeoDataFrame(zones_df, crs=default_crs)

In [ ]:
weight = "length (m)"
# weight = "flow_time (s)"
all_paths, new_graph = run_naive_simulation(G, trip_mat, weight=weight)
for i in range(len(all_paths)):
    for j in range(len(all_paths)):
        assert len(all_paths[i][j]) == trip_mat[i, j]

In [ ]:
ts = int(time.time())
map = draw_trips_map(new_graph)
map.save(os.path.join(html_dir, f"paths_{ts}.html"))

with open(os.path.join(data_dir, f"paths_by_len_{ts}.pkl"), "wb") as f:
    pickle.dump(all_paths, f)

In [ ]:
# weight = "length (m)"
weight = "flow_time (s)"
all_paths, new_graph = run_naive_simulation(G, trip_mat, weight=weight)
for i in range(len(all_paths)):
    for j in range(len(all_paths)):
        assert len(all_paths[i][j]) == trip_mat[i, j]

In [ ]:
ts = int(time.time())
map = draw_trips_map(new_graph)
map.save(os.path.join(html_dir, f"paths_{ts}.html"))

with open(os.path.join(data_dir, f"paths_by_time_{ts}.pkl"), "wb") as f:
    pickle.dump(all_paths, f)